In [0]:
import pandas as pd
import numpy as np 
import os
import math
from collections import defaultdict
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.utils import np_utils
from keras.preprocessing.sequence import pad_sequences
from keras.layers import SpatialDropout1D, LSTM, BatchNormalization,concatenate,Flatten,Embedding,Dense,Dropout,MaxPooling2D,Reshape,CuDNNLSTM
from keras.models import Sequential
from keras import Model,Input
from keras.layers.convolutional import Conv2D,Conv1D
import keras.backend as k
from sklearn.metrics import roc_auc_score
import tensorflow as tf
import keras
from sklearn.utils import compute_class_weight
from keras.initializers import he_normal,glorot_normal
from keras.regularizers import l1,l2
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint,LearningRateScheduler
from time import time
from tensorflow.python.keras.callbacks import TensorBoard
from keras.preprocessing.sequence import pad_sequences
from IPython.display import SVG, display
import pickle 
import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [0]:
%tensorflow_version 2.x
import tensorflow as tf
import string
import requests

TensorFlow is already loaded. Please restart the runtime to change versions.


# Source: https://github.com/laxmimerit/NLP-Tutorial-11---Automatic-Text-Generation-using-TensorFlow-Keras-and-LSTM/blob/master/Automatic_Text_Generation_Using_Keras_and_LSTM.ipynb

In [0]:
response = requests.get('https://ocw.mit.edu/ans7870/6/6.006/s08/lecturenotes/files/t8.shakespeare.txt')

In [0]:
# response.text

In [0]:
data=response.text.split('\n')
data[0]

'This is the 100th Etext file presented by Project Gutenberg, and'

In [0]:
len(data)

124457

In [0]:
data = " ".join(data)

In [0]:
def clean_text(doc):
  tokens = doc.split()
  table = str.maketrans('', '', string.punctuation)
  tokens = [w.translate(table) for w in tokens]
  tokens = [word for word in tokens if word.isalpha()]
  tokens = [word.lower() for word in tokens]
  return tokens

In [0]:
tokens = clean_text(data)
print(tokens[:50])

['this', 'is', 'the', 'etext', 'file', 'presented', 'by', 'project', 'gutenberg', 'and', 'is', 'presented', 'in', 'cooperation', 'with', 'world', 'library', 'inc', 'from', 'their', 'library', 'of', 'the', 'future', 'and', 'shakespeare', 'cdroms', 'project', 'gutenberg', 'often', 'releases', 'etexts', 'that', 'are', 'not', 'placed', 'in', 'the', 'public', 'domain', 'shakespeare', 'this', 'etext', 'has', 'certain', 'copyright', 'implications', 'you', 'should', 'read']


In [0]:
len(tokens)

899788

In [0]:
len(set(tokens))

28113

In [0]:
length = 50 + 1
lines = []

for i in range(length, len(tokens)):
  seq = tokens[i-length:i]
  line = ' '.join(seq)
  lines.append(line)
  if i > 200000:
    break

print(len(lines))

199951


In [0]:
lines[0]

'this is the etext file presented by project gutenberg and is presented in cooperation with world library inc from their library of the future and shakespeare cdroms project gutenberg often releases etexts that are not placed in the public domain shakespeare this etext has certain copyright implications you should read this'

In [0]:
tokens[50]

'this'

In [0]:
lines[1]

'is the etext file presented by project gutenberg and is presented in cooperation with world library inc from their library of the future and shakespeare cdroms project gutenberg often releases etexts that are not placed in the public domain shakespeare this etext has certain copyright implications you should read this electronic'

In [0]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [0]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [0]:
sequences = np.array(sequences)
X, y = sequences[:, :-1], sequences[:,-1]

In [0]:
len(X[10])

50

In [0]:
X[0]

array([   22,    11,     1,   397,  2006,  6743,    30,   398,   455,
           2,    11,  6743,    10,  6744,    15,   122,   486,   520,
          47,    65,   486,     5,     1,  2775,     2,   418, 13163,
         398,   455,   678, 13162,  2187,     9,    33,    12,  4811,
          10,     1,   848,  3823,   418,    22,   397,   236,   611,
         485, 13161,     6,    86,   610])

In [0]:
vocab_size = len(tokenizer.word_index) + 1

In [0]:
y = to_categorical(y, num_classes=vocab_size)

In [0]:
seq_length = X.shape[1]

In [0]:
seq_length

50

In [0]:
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 50)            658250    
_________________________________________________________________
lstm (LSTM)                  (None, 50, 100)           60400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 13165)             1329665   
Total params: 2,138,815
Trainable params: 2,138,815
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [0]:
model.fit(X, y, batch_size = 256, epochs = 50)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 199951 samples
Epoch 1/50
199951/199951 [==============================] - 139s 696us/sample - loss: 6.9053 - acc: 0.0298
Epoch 2/50
199951/199951 [==============================] - 137s 686us/sample - loss: 6.5710 - acc: 0.0396
Epoch 3/50
199951/199951 [==============================] - 138s 688us/sample - loss: 6.4226 - acc: 0.0488
Epoch 4/50
199951/199951 [==============================] - 137s 685us/sample - loss: 6.2382 - acc: 0.0656
Epoch 5/50
199951/199951 [==============================] - 137s 685us/sample - loss: 6.0880 - acc: 0.0770
Epoch 6/50
199951/199951 [==============================] - 137s 684us/sample - loss: 5.9705 - acc: 0.0844
Epoch 7/50
199951/199951 [==============================] - 137s 685us/sample - loss: 5.8694 - acc: 0.0903
Epoch 8/50
199951/199951 [==============================] - 136s 679us/sample - loss: 5.7745 - acc: 0.0961
Epoch 9/50
199951/199951 [

In [0]:
seed_text='' + lines[12343]

In [0]:
def generate_text_seq(model, tokenizer, text_seq_length, seed_text, n_words):
  text = []

  for _ in range(n_words):
    encoded = tokenizer.texts_to_sequences([seed_text])[0]
    encoded = pad_sequences([encoded], maxlen = text_seq_length, truncating='pre')

    y_predict = model.predict_classes(encoded)

    predicted_word = ''
    for word, index in tokenizer.word_index.items():
      if index == y_predict:
        predicted_word = word
        break
    seed_text = seed_text + ' ' + predicted_word
    text.append(predicted_word)
  return ' '.join(text)

In [0]:
# generate_text_seq(model, tokenizer, seq_length, seed_text, 100)

In [0]:
# seed_text

In [41]:
print("Seed Text : ")
print(seed_text)
print("-"*80)
print("Generated Text : ")
print(generate_text_seq(model, tokenizer, seq_length, seed_text, 100))

Seed Text : 
cannot dispraise but in a kind of praise naming thy name blesses an ill report o what a mansion have those vices got which for their habitation chose out thee where beautys veil doth cover every blot and all things turns to fair that eyes can see take heed dear heart
--------------------------------------------------------------------------------
Generated Text
i am forsaken a torment thrice threefold cupids for his preferment fault to the people as he is not a man that being so allowd to violate my mother bows at the castle enter king and attendants and others a print of the forest enter a messenger with a soninlaw pardons the chain of my heart and to the people adriana o wonderful a man and thought to make his heart and yet the chests and guildenstern and undoubted epilogue army i have seen the people of the people of the people bonneted of it hereafter but i have seen


In [44]:
model.fit(X, y, batch_size = 256, epochs = 200)

Train on 199951 samples
Epoch 1/200
199951/199951 [==============================] - 136s 682us/sample - loss: 3.9298 - acc: 0.2344
Epoch 2/200
199951/199951 [==============================] - 136s 682us/sample - loss: 3.9036 - acc: 0.2377
Epoch 3/200
199951/199951 [==============================] - 136s 681us/sample - loss: 3.8811 - acc: 0.2420
Epoch 4/200
199951/199951 [==============================] - 136s 680us/sample - loss: 3.8621 - acc: 0.2432
Epoch 5/200
199951/199951 [==============================] - 137s 686us/sample - loss: 3.8432 - acc: 0.2466
Epoch 6/200
199951/199951 [==============================] - 136s 682us/sample - loss: 3.8186 - acc: 0.2502
Epoch 7/200
199951/199951 [==============================] - 137s 685us/sample - loss: 3.7995 - acc: 0.2523
Epoch 8/200
199951/199951 [==============================] - 137s 683us/sample - loss: 3.7791 - acc: 0.2551
Epoch 9/200
199951/199951 [==============================] - 137s 685us/sample - loss: 3.7583 - acc: 0.2583
Epoc

In [46]:
print("Seed Text : ")
print(seed_text)
print("-"*80)
print("Generated Text : ")
print(generate_text_seq(model, tokenizer, seq_length, seed_text, 100))

Seed Text : 
cannot dispraise but in a kind of praise naming thy name blesses an ill report o what a mansion have those vices got which for their habitation chose out thee where beautys veil doth cover every blot and all things turns to fair that eyes can see take heed dear heart
--------------------------------------------------------------------------------
Generated Text : 
she stole to quit the ring and with feasts took couched and he will shine me against me which cupid got them as mens betraying us i have looked for myself and that of ills depends upon my closet with your pledge into offer for friends and loss to marvells dolorous to email distills your wound to woo me but as i was not to know it is the first time ill dwell to their whipping i would have found it diana i am not hurt me i will merrily accompany you to the shouting plebeians brutus i have taken


In [0]:
# # model = Sequential()
# # model.add(Embedding(vocab_size, 50, input_length=seq_length))
# # model.add(LSTM(100, return_sequences=True))
# # model.add(LSTM(100))
# # model.add(Dense(100, activation='relu'))
# # model.add(Dense(vocab_size, activation='softmax'))

# model = Sequential()
# model.add(LSTM(256, input_shape=(vocab_size, 50), return_sequences=True))
# model.add(Dropout(0.2))
# model.add(LSTM(256))
# model.add(Dropout(0.2))
# model.add(Dense(vocab_size, activation='softmax'))
# model.compile(loss='categorical_crossentropy', optimizer='adam')

In [0]:
# model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [0]:
# model.fit(X, y, batch_size = 256, epochs = 100)